# Required libraries

In [ ]:
!huggingface-cli login

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio

In [ ]:
!pip install peft

In [ ]:
!pip install langchain faiss-cpu sentence-transformers

In [ ]:
!pip install -U langchain-community

#Rag Database

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


In [ ]:
def get_full_rag_content(query, k=3):
    """Get complete content from RAG for specific queries"""

    print(f"🔍 SEARCHING RAG FOR: '{query}'")
    print("=" * 60)

    try:
        # Search your vectorstore
        results = vectorstore.similarity_search(query, k=k)

        if results:
            print(f"✅ Found {len(results)} relevant documents")

            for i, result in enumerate(results):
                print(f"\n📄 DOCUMENT {i+1}:")
                print("-" * 40)
                print(f"FULL CONTENT ({len(result.page_content)} characters):")
                print(result.page_content)
                print("-" * 40)

                # Check for specific keywords
                keywords_found = []
                search_terms = query.lower().split()

                for term in search_terms:
                    if term in result.page_content.lower():
                        keywords_found.append(term)

                print(f"📊 Keywords found: {keywords_found}")
                print(f"📊 Relevance: {len(keywords_found)}/{len(search_terms)} terms matched")

        else:
            print("❌ No results found for this query")

    except Exception as e:
        print(f"❌ Search error: {e}")

# Test with your specific example
# print("🎯 TESTING SPECIFIC QUERY:")
# get_full_rag_content("zone minutes 42 year old female")

print("\n" + "="*80)

# Test some other relevant queries
test_queries = [
    "exercise recommendations 25 year old",
    # "physical activity middle aged women",
    # "cardio minutes weekly adults",
    # "moderate intensity exercise adults",
    # "150 minutes physical activity"
]

for query in test_queries:
    print(f"\n{'='*80}")
    get_full_rag_content(query, k=2)  # Get top 2 results for each

# Data structure and Memory System

In [ ]:
import torch
import json
import os
from datetime import datetime # Import datetime
from typing import Dict, List
import gradio as gr

class SimpleDataManager:
    """Simple file-based storage for user data and recommendations"""

    def __init__(self, data_dir="wellbeing_data"):
        self.data_dir = data_dir
        os.makedirs(data_dir, exist_ok=True)
        os.makedirs(f"{data_dir}/users", exist_ok=True)
        os.makedirs(f"{self.data_dir}/recommendations", exist_ok=True) # Corrected path

    def save_user_week(self, user_id: str, week_data: Dict):
        """Save weekly data for a user"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            # Load existing data
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
            else:
                user_data = {"user_id": user_id, "weeks": []}

            # Add new week
            user_data["weeks"].append(week_data)

            # Keep only last 8 weeks
            user_data["weeks"] = user_data["weeks"][-8:]

            # Save
            with open(user_file, 'w') as f:
                json.dump(user_data, f, indent=2)

            print(f"✅ Saved week data for user {user_id}")

        except Exception as e:
            print(f"❌ Error saving user data: {str(e)}")

    def get_user_history(self, user_id: str) -> List[Dict]:
        """Get user's weekly history"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
                    history = user_data.get("weeks", [])
                    print(f"📊 Retrieved {len(history)} weeks of history for {user_id}")
                    return history
        except Exception as e:
            print(f"❌ Error loading user history: {str(e)}")

        return []

    def save_recommendation(self, user_id: str, week_start: str, recommendation: str):
        """Save LLM recommendation"""
        # Clean week_start for filename (remove invalid characters)
        clean_week = week_start.replace("/", "-").replace(":", "-")
        rec_file = f"{self.data_dir}/recommendations/{user_id}_{clean_week}.json"

        try:
            rec_data = {
                "user_id": user_id,
                "week_start": week_start,
                "recommendation": recommendation,
                "timestamp": datetime.now().isoformat()
            }

            with open(rec_file, 'w') as f:
                json.dump(rec_data, f, indent=2)

            print(f"✅ Saved recommendation for user {user_id}, week {week_start}")

        except Exception as e:
            print(f"❌ Error saving recommendation: {str(e)}")

    def get_last_recommendation(self, user_id: str) -> str:
        """Get user's last recommendation"""
        rec_dir = f"{self.data_dir}/recommendations"

        try:
            # Check if recommendations directory exists
            if not os.path.exists(rec_dir):
                return ""

            # Find latest recommendation file for this user
            user_files = [f for f in os.listdir(rec_dir) if f.startswith(f"{user_id}_") and f.endswith('.json')]

            if user_files:
                # Sort by date and get latest
                user_files.sort(reverse=True)
                latest_file = f"{rec_dir}/{user_files[0]}"

                with open(latest_file, 'r') as f:
                    rec_data = json.load(f)
                    recommendation = rec_data.get("recommendation", "")
                    print(f"📝 Retrieved last recommendation for {user_id} ({len(recommendation)} chars)")
                    return recommendation

        except Exception as e:
            print(f"❌ Error loading last recommendation: {str(e)}")

        return ""

    def get_user_stats(self, user_id: str) -> Dict:
        """Get basic stats about user's data (bonus method)"""
        history = self.get_user_history(user_id)

        if not history:
            return {"total_weeks": 0}

        return {
            "total_weeks": len(history),
            "first_week": history[0].get("week_start", "Unknown"),
            "latest_week": history[-1].get("week_start", "Unknown"),
            "avg_steps": sum(week.get("total_steps", 0) for week in history) // len(history),
            "avg_sleep": sum(week.get("avg_sleep", 0) for week in history) / len(history)
        }

#Well being LLM

In [ ]:
import torch
import re
from typing import Dict, List


class WellbeingLLM:
    """Improved LLM system for wellbeing recommendations with better prompt engineering"""

    def __init__(self, base_model_id: str, adapter_path: str, vectorstore_path: str = None):
        self.model = None
        self.tokenizer = None
        self.device = None
        self.vectorstore = None

        # Load vectorstore if path provided
        if vectorstore_path:
            self.load_vectorstore(vectorstore_path)

        self.load_model(base_model_id, adapter_path)

    def load_vectorstore(self, vectorstore_path: str):
        """Load FAISS vectorstore for RAG"""
        try:
            from langchain.vectorstores import FAISS
            from langchain.embeddings import HuggingFaceEmbeddings

            print("📚 Loading knowledge base...")
            embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

            self.vectorstore = FAISS.load_local(
                vectorstore_path,
                embeddings=embedding_model,
                allow_dangerous_deserialization=True
            )
            print("✅ Knowledge base loaded")

        except Exception as e:
            print(f"⚠️ Could not load vectorstore: {e}")
            self.vectorstore = None

    def load_model(self, base_model_id: str, adapter_path: str = None):
        """Load the LLM model with better error handling"""
        try:
            print("🤖 Loading LLM...")

            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
            from peft import PeftModel

            # Clear GPU memory and check device
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                print(f"🔍 GPU available: {torch.cuda.get_device_name()}")
            else:
                print("🔍 Using CPU")

            # Load tokenizer
            print("📝 Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer loaded")

            # Quantization config
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=False,
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            print("⚙️ Quantization config ready")

            # Load base model
            print("🧠 Loading base model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model_id,
                device_map="auto",
                torch_dtype=torch.float16,  # Faster than float16
                quantization_config=bnb_config,
                trust_remote_code=True,
                low_cpu_mem_usage=True,      # 🚀 NEW: Reduces CPU memory usage
                use_cache=False,             # 🚀 NEW: Faster loading
            )
            print("✅ Base model loaded")

            # Load fine-tuned adapter
            # print("🎯 Loading fine-tuned adapter...")
            # self.model = PeftModel.from_pretrained(self.model, adapter_path)
            self.model.eval()

            # Store device for later use
            self.device = next(self.model.parameters()).device
            print(f"✅ LLM loaded successfully on {self.device}")

        except Exception as e:
            print(f"❌ LLM loading failed: {str(e)}")
            import traceback
            traceback.print_exc()
            self.model = None
            self.tokenizer = None
            self.device = None




    def retrieve_relevant_context(self, current_week: Dict) -> Dict[str, str]:
        """Retrieve relevant information from vectorstore - IMPROVED"""

        print("🔍 DEBUG - RAG Context being built from:")
        print(f"   Diet type: {current_week.get('preferences', {}).get('diet_type', 'MISSING')}")
        print(f"   Total steps: {current_week.get('total_steps', 'MISSING')}")

        if not self.vectorstore:
            return {}

        try:
            # Build targeted search queries
            total_steps = current_week.get('total_steps', 0)
            zone_minutes = current_week.get('zone_minutes', 0)
            avg_sleep = current_week.get('avg_sleep', 0)
            diet_type = current_week.get('preferences', {}).get('diet_type', 'No Preference')

            # Create category-specific queries
            category_contexts = {}

            # Food context
            if diet_type != 'No Preference':
                food_docs = self.vectorstore.similarity_search(f"{diet_type} nutrition guidelines", k=1)
                if food_docs:
                    category_contexts['food'] = food_docs[0].page_content[:300]

            # Add this to your function
            allergies = current_week.get('preferences', {}).get('allergies', 'None')
            if allergies != 'None':
                allergy_docs = self.vectorstore.similarity_search(f"{allergies} allergy foods avoid", k=2)
                if allergy_docs:
                    category_contexts['allergies'] = allergy_docs[0].page_content[:300]  # More content

            # Always get basic nutrition info
            nutrition_docs = self.vectorstore.similarity_search("daily protein carbohydrate requirements", k=2)
            if nutrition_docs:
                category_contexts['nutrition'] = nutrition_docs[0].page_content[:300]

            # Exercise context
            if total_steps < 70000 or zone_minutes < 150:
                exercise_docs = self.vectorstore.similarity_search("exercise recommendations cardio fitness", k=1)
                if exercise_docs:
                    category_contexts['exercise'] = exercise_docs[0].page_content[:150]

            # Sleep context
            if avg_sleep < 7.5:
                sleep_docs = self.vectorstore.similarity_search("sleep hygiene improvement tips", k=1)
                if sleep_docs:
                    category_contexts['sleep'] = sleep_docs[0].page_content[:150]

            print(f"📚 Retrieved {len(category_contexts)} category-specific contexts")
            return category_contexts

        except Exception as e:
            print(f"⚠️ Error retrieving context: {e}")
            return {}

    def generate_recommendation(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
      """Generate personalized recommendations with single template enforcement"""

      if not self.model or not self.tokenizer:
          print("❌ No model loaded")
          return "Model not available. Please check model loading."

      max_retries = 3

      for attempt in range(max_retries):
          try:
              print(f"🔍 Attempt {attempt + 1} - Using single template strategy...")

              # Use the single master template
              prompt = self._build_master_template(current_week, user_history)
              print(f"🔍 Using master template ({len(prompt)} chars)")

              inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2000)
              inputs = {k: v.to(self.device) for k, v in inputs.items()}

              with torch.no_grad():
                  outputs = self.model.generate(
                      **inputs,
                      max_new_tokens=500,
                      min_new_tokens=150,
                      temperature=0.7,  # Low temperature for consistency
                      top_p=0.9,
                      do_sample=True,
                      repetition_penalty=1.15,
                      early_stopping=False,
                      pad_token_id=self.tokenizer.eos_token_id,
                      eos_token_id=self.tokenizer.eos_token_id,
                      use_cache=True
                  )

              full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
              response = full_response[len(prompt):].strip()

              # Clean response
              response = self._clean_response(response)

              # Validate structure
              if self._validate_template_structure(response):
                  print(f"✅ Template followed successfully on attempt {attempt + 1}")
                  return response
              else:
                  print(f"❌ Template validation failed on attempt {attempt + 1}")

          except Exception as e:
              print(f"❌ Generation failed on attempt {attempt + 1}: {str(e)}")

      # If all attempts fail, return the last attempt anyway (LLM-generated)
      print("⚠️ Returning best attempt even if template isn't perfect")
      return response if 'response' in locals() else "Unable to generate recommendation."

    def _build_master_template(self, current_week: Dict, user_history: List[Dict]) -> str:
      """Single master template with RAG-enhanced knowledge"""

      # Extract user data
      total_steps = current_week.get('total_steps', 0)
      zone_minutes = current_week.get('zone_minutes', 0)
      demographics = current_week.get('demographics', {})
      age = demographics.get('age', 'unknown')
      weight = demographics.get('weight', 'unknown')
      height = demographics.get('height', 'unknown')
      sex = demographics.get('sex', 'unknown')
      avg_sleep = current_week.get('avg_sleep', 0)
      diet_type = current_week.get('preferences', {}).get('diet_type', 'No Preference')
      allergies = current_week.get('preferences', {}).get('allergies', [])
      exercise_sessions = current_week.get('exercise_sessions', 0)

      print("🔍 DEBUG - Input data received:")
      print(f"   Demographics: {current_week.get('demographics', 'MISSING')}")
      print(f"   Steps: {current_week.get('total_steps', 'MISSING')}")
      print(f"   Zone minutes: {current_week.get('zone_minutes', 'MISSING')}")
      print(f"   Preferences: {current_week.get('preferences', 'MISSING')}")

      # Build progress context
      progress_info = self._get_progress_analysis(current_week, user_history)
      specific_needs = self._identify_specific_needs(current_week)

      # 🚀 ACTUALLY Retrieve RAG context
      rag_contexts = self.retrieve_relevant_context(current_week)
      print(f"📚 Retrieved RAG contexts: {list(rag_contexts.keys())}")

      # 🚀 Format RAG context for LLM guidance
      expert_knowledge = ""
      if rag_contexts:
          expert_knowledge = "\n📚 EXPERT KNOWLEDGE BASE (use this information in your recommendations):\n"

          if 'food' in rag_contexts:
              expert_knowledge += f"🍎 NUTRITION: {rag_contexts['food']}\n"

          if 'exercise' in rag_contexts:
              expert_knowledge += f"🏃 EXERCISE: {rag_contexts['exercise']}\n"

          if 'sleep' in rag_contexts:
              expert_knowledge += f"😴 SLEEP: {rag_contexts['sleep']}\n"

          if 'allergies' in rag_contexts:
              expert_knowledge += f"⚠️ ALLERGIES: {rag_contexts['allergies']}\n"

          if 'nutrition' in rag_contexts:
              expert_knowledge += f"📊 REQUIREMENTS: {rag_contexts['nutrition']}\n"

      # The master template with RAG integration
      prompt = f"""You are a wellness coach AI. Use the expert knowledge provided to give evidence-based recommendations.

  USER DATA:
  Age: {age} years
  Sex: {sex}
  Weight: {weight}kg
  Height: {height}cm
  Steps: {total_steps:,}/70,000 weekly target
  Zone Minutes: {zone_minutes}/150 minutes weekly target
  Exercise Sessions: {exercise_sessions} weekly
  Sleep: {avg_sleep:.1f} hours nightly
  Diet preference: {diet_type}
  Allergies: {', '.join(allergies) if allergies else 'None'}

  {progress_info}

  AREAS NEEDING ATTENTION:
  {specific_needs}

  {expert_knowledge}

  INSTRUCTIONS:
  - Use the expert knowledge above to provide specific, evidence-based recommendations
  - Follow the exact template structure below
  - Give realistic numbers based on the expert knowledge (e.g., if expert knowledge says "225-325g carbs", use those ranges)
  - Consider the user's allergies and diet preferences in all recommendations

  MANDATORY DATA (DO NOT CHANGE):
  - User age: EXACTLY {age} years old
  - Current steps: EXACTLY {total_steps:,} weekly
  - Current zone minutes: EXACTLY {zone_minutes} weekly
  - Target zone minutes: EXACTLY 150 weekly (WHO standard)

  MANDATORY RESPONSE FORMAT:
  **1) Food Recommendation**
  a) Overall Assessment: [Use expert nutrition knowledge to assess their current needs]
  b) Areas to Improve: [Specific improvements based on expert guidelines]
  c) Meal and Food Suggestions: [Specific foods considering diet type and allergies]

  **2) Physical Exercise**
  a) Activity and Performance Analysis: [Compare their {total_steps} steps to expert recommendations]
  b) Zone Minutes and Intensity Feedback: [Compare their {zone_minutes} minutes to 150-minute target]
  c) Strength Training Analysis: [Based on expert exercise knowledge]
  d) Cardio Suggestions: [Specific cardio activities from expert knowledge]

  **3) Sleep and Social Well-being**
  a) Sleep/Mood Analysis: [Assess their {avg_sleep:.1f} hours against expert sleep guidelines]
  b) Suggestions: [Specific sleep improvements from expert knowledge]

  **4) Overall Weekly Recommendation**
  a) Summary: [Overall assessment using expert knowledge]
  b) Weekly Goals: [Specific, measurable goals based on expert recommendations]

  Generate response following this exact structure:"""

      return prompt

    def _validate_template_structure(self, response: str) -> bool:
        """Validate the response follows the template structure"""

        # Check for main sections
        required_sections = [
        "**1) Food Recommendation**",
        "**2) Physical Exercise**",
        "**3) Sleep and Social Well-being**",
        "**4) Overall Weekly Recommendation**"
    ]

        sections_found = 0
        for section in required_sections:
            if section in response:
                sections_found += 1
            else:
                print(f"❌ Missing section: {section}")

        # Check for required bullet points in each section
        bullet_points_found = response.count('- Specific vegetables to try:')
        bullet_points_found += response.count('- Legumes and proteins:')
        bullet_points_found += response.count('- Meal suggestions:')
        bullet_points_found += response.count('- Cardio activities:')
        bullet_points_found += response.count('- Strength exercises:')
        bullet_points_found += response.count('- Sleep tips:')
        bullet_points_found += response.count('- Wellness activities:')
        bullet_points_found += response.count('- Top priorities:')
        bullet_points_found += response.count('- Weekly goals:')

        print(f"🔍 Found {sections_found}/4 sections and {bullet_points_found}/9 bullet points")

        # Consider valid if we have at least 3 sections and 6 bullet points
        return sections_found >= 3 and bullet_points_found >= 6




    def _identify_specific_needs(self, current_week: Dict) -> str:
        """Identify specific areas needing attention"""
        needs = []

        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)
        food_data = current_week.get('food_data', {})

        # Activity needs
        if total_steps < 70000:
            deficit = 70000 - total_steps
            needs.append(f"ACTIVITY: Add {deficit:,} more steps ({deficit//7:,} per day)")

        if zone_minutes < 150:
            deficit = 150 - zone_minutes
            needs.append(f"CARDIO: Add {deficit} more zone minutes")

        # Sleep needs
        if avg_sleep < 7:
            needs.append(f"SLEEP: Increase by {7 - avg_sleep:.1f} hours nightly")

        # Nutrition needs
        water = food_data.get('water_liters', 0)
        # if water < 14:
        #     needs.append(f"HYDRATION: Drink {14 - water:.1f}L more water weekly")

        vegetables = food_data.get('vegetables_grams', 0)
        # if vegetables < 1750:
        #     needs.append(f"NUTRITION: Add {1750 - vegetables:.0f}g more vegetables weekly")

        return '\n'.join(needs) if needs else "Maintaining current healthy patterns"

    def _get_progress_analysis(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Analyze progress for personalized feedback"""
        if len(user_history) < 2:
            return "PROGRESS: First week tracked"

        prev_week = user_history[-2]

        changes = []

        # Step progress
        step_change = current_week.get('total_steps', 0) - prev_week.get('total_steps', 0)
        if step_change > 0:
            changes.append(f"Steps IMPROVED by {step_change:,}")
        elif step_change < 0:
            changes.append(f"Steps DECREASED by {abs(step_change):,}")

        # Zone minutes progress
        zone_change = current_week.get('zone_minutes', 0) - prev_week.get('zone_minutes', 0)
        if zone_change > 0:
            changes.append(f"Cardio IMPROVED by {zone_change} minutes")
        elif zone_change < 0:
            changes.append(f"Cardio DECREASED by {abs(zone_change)} minutes")

        # Sleep progress
        sleep_change = current_week.get('avg_sleep', 0) - prev_week.get('avg_sleep', 0)
        if sleep_change > 0.2:
            changes.append(f"Sleep IMPROVED by {sleep_change:.1f}h")
        elif sleep_change < -0.2:
            changes.append(f"Sleep DECREASED by {abs(sleep_change):.1f}h")

        progress_text = f"PROGRESS FROM LAST WEEK:\n{chr(10).join(changes)}" if changes else "PROGRESS: Similar to last week"
        return progress_text








    def _clean_response(self, response: str) -> str:
        """Clean the generated response"""
        # Remove unwanted prefixes
        response = re.sub(r'^(Response:|Recommendations?:)', '', response, flags=re.IGNORECASE)
        response = response.strip()

        # Remove excessive newlines
        response = re.sub(r'\n{3,}', '\n\n', response)

        # Remove repetitive phrases
        response = re.sub(r'(Based on your.*?)\1+', r'\1', response, flags=re.IGNORECASE)

        return response.strip()


# Well being system

In [ ]:
class WellbeingSystem:
    """Main system that combines data management and LLM"""

    def __init__(self, base_model_id: str, adapter_path: str, vectorstore_path: str = None):
        self.data_manager = SimpleDataManager()
        self.llm = WellbeingLLM(base_model_id, adapter_path, vectorstore_path) # Pass vectorstore_path to LLM
        print("🎯 WellbeingSystem initialized")

    def analyze_and_recommend(self, user_id: str, week_data: Dict) -> str:
        """Main function: analyze weekly data and generate recommendations"""

        print(f"📊 Analyzing data for user: {user_id}")

        try:
            # Get user history and last recommendation
            user_history = self.data_manager.get_user_history(user_id)
            last_recommendation = self.data_manager.get_last_recommendation(user_id)

            print(f"📈 Found {len(user_history)} weeks of history")
            if last_recommendation:
                print(f"📝 Previous recommendation found ({len(last_recommendation)} chars)")

            # Process current week data
            processed_week = self._process_week_data(week_data)

            # Show what data is being sent to LLM
            print(f"📊 Processed data: {processed_week['total_steps']:,} steps, {processed_week['zone_minutes']} zone mins, {processed_week['avg_sleep']:.1f}h sleep")

            # Generate recommendation using LLM
            recommendation = self.llm.generate_recommendation(
                processed_week, user_history, last_recommendation
            )

            # Save data (with error handling)
            try:
                self.data_manager.save_user_week(user_id, processed_week)
                self.data_manager.save_recommendation(user_id, processed_week['week_start'], recommendation)
                print(f"💾 Data saved successfully")
            except Exception as save_error:
                print(f"⚠️ Warning: Could not save data: {save_error}")
                # Continue anyway - return the recommendation even if saving fails

            print(f"✅ Generated recommendation for {user_id} ({len(recommendation)} chars)")
            return recommendation

        except Exception as e:
            print(f"❌ Error in analyze_and_recommend: {str(e)}")
            import traceback
            traceback.print_exc()
            return f"Analysis failed: {str(e)}. Please check your input data."

    def _process_week_data(self, week_data: Dict) -> Dict:
        """Process and standardize weekly data with better validation"""

        try:
            # Parse sleep and mood data with validation
            sleep_hours = week_data.get('sleep_hours', [])
            mood_scores = week_data.get('mood_scores', [])

            # Handle string input (comma-separated values)
            if isinstance(sleep_hours, str):
                try:
                    sleep_hours = [float(x.strip()) for x in sleep_hours.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid sleep hours format, using default")
                    sleep_hours = []

            if isinstance(mood_scores, str):
                try:
                    mood_scores = [float(x.strip()) for x in mood_scores.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid mood scores format, using default")
                    mood_scores = []

            # Calculate averages with validation
            avg_sleep = sum(sleep_hours) / len(sleep_hours) if sleep_hours else 0
            avg_mood = sum(mood_scores) / len(mood_scores) if mood_scores else 0

            # 🚀 ADD DEBUG:
            print(f"🔍 DEBUG - Sleep calculation: {sleep_hours} → avg: {avg_sleep}")
            print(f"🔍 DEBUG - Mood calculation: {mood_scores} → avg: {avg_mood}")

            # Enhanced food data processing
            food_data = self._process_food_data(week_data)

            processed_data = {
                'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                'total_steps': max(0, int(week_data.get('total_steps', 0))), # Ensure int and non-negative
                'zone_minutes': max(0, int(week_data.get('zone_minutes', 0))), # Ensure int and non-negative
                'exercise_sessions': max(0, int(week_data.get('exercise_sessions', 0))), # Ensure int and non-negative
                'avg_sleep': round(avg_sleep, 1),
                'avg_mood': round(avg_mood, 1),
                'food_data': food_data,
                'timestamp': datetime.now().isoformat(),
                'demographics': week_data.get('demographics', {})
            }

            # Include preferences and targets if present in the input week_data
            if 'preferences' in week_data:
                processed_data['preferences'] = week_data['preferences']
            if 'targets' in week_data:
                processed_data['targets'] = week_data['targets']


            return processed_data

        except Exception as e:
            print(f"❌ Error processing week data: {str(e)}")
            # Return empty or default data structure on error
            return {
                 'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                 'total_steps': 0,
                 'zone_minutes': 0,
                 'exercise_sessions': 0,
                 'avg_sleep': 0.0,
                 'avg_mood': 0.0,
                 'food_data': {},
                 'timestamp': datetime.now().isoformat(),
                 'processing_error': str(e)
            }


    def _process_food_data(self, week_data: Dict) -> Dict:
        """Process food data handling both old and new formats"""

        food_data = {}

        # Check for new format first (with units)
        unit_fields = ['dairy_liters', 'water_liters', 'legumes_grams', 'meat_grams',
                      'fruits_grams', 'vegetables_grams', 'grains_grams', 'nuts_seeds_grams']

        has_unit_format = any(field in week_data for field in unit_fields)

        if has_unit_format:
            print("📊 Using new format (with units)")
            # New format with units
            for field in unit_fields:
                value = week_data.get(field, 0)
                try:
                    food_data[field] = max(0, float(value))  # Ensure non-negative
                except (ValueError, TypeError):
                    food_data[field] = 0
                    print(f"⚠️ Invalid {field} value, using 0")
        else:
            print("📊 Using old format (servings)")
            # Old format (for backward compatibility)
            old_fields = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']
            for field in old_fields:
                if field in week_data:
                    try:
                        food_data[field] = max(0, float(week_data[field]))
                    except (ValueError, TypeError):
                        food_data[field] = 0

        return food_data





# Well being app

In [ ]:
import gradio as gr
import json
import os
from datetime import datetime
from typing import Dict, List

class WellbeingApp:
    """Complete Gradio interface with targets, history, and preferences"""

    def __init__(self, base_model_id: str, adapter_path: str):
      print("🚀 Initializing Complete WellbeingApp...")
      self.wellbeing_system = WellbeingSystem(
          base_model_id,
          adapter_path,
          vectorstore_path="/content/drive/MyDrive/rag_index"
      )

      # Default targets (same as before)
      self.default_targets = {
          'weekly_steps': 70000,
          'weekly_zone_minutes': 150,
          'daily_sleep_hours': 8.0,
          'weekly_water_liters': 14.0,
          'weekly_fruits_grams': 2100,
          'weekly_vegetables_grams': 3500,
          'weekly_exercise_sessions': 4
      }

    def create_wellbeing_app(self):
        """Create comprehensive Gradio interface with tabs"""

        with gr.Blocks(title="🏥 Complete Wellbeing LLM", theme=gr.themes.Soft()) as demo:

            gr.Markdown("# 🏥 Complete AI Wellbeing System")
            gr.Markdown("### Personalized health recommendations with custom targets and progress tracking")

            # Create tabs for different sections
            with gr.Tabs():

                # TAB 1: MAIN ANALYSIS
                with gr.TabItem("🤖 Get Recommendations", elem_id="main-tab"):
                    self._create_main_analysis_tab()

                # TAB 2: TARGETS CUSTOMIZATION
                with gr.TabItem("🎯 Customize Targets", elem_id="targets-tab"):
                    self._create_targets_tab()

                # TAB 3: HISTORY REVIEW
                with gr.TabItem("📈 Progress History", elem_id="history-tab"):
                    self._create_history_tab()

                # TAB 4: SYSTEM INFO
                # with gr.TabItem("ℹ️ System Info", elem_id="info-tab"):
                #     self._create_info_tab()

        return demo

    def _create_main_analysis_tab(self):
      """Create the main analysis tab with fixed component handling"""

      with gr.Row():
          with gr.Column():

              gr.Markdown("### 🤖 AI Enhancement Options")
              # User Info Section
              gr.Markdown("### 👤 User Information")
              user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Unique identifier for tracking your progress")
              week_start = gr.Textbox(label="📅 Week Start Date", value="2024-01-15", info="Format: YYYY-MM-DD")

              # Personal Details - FIXED: These were not being collected properly
              sex = gr.Dropdown(
                  label="⚧️ Sex",
                  choices=["Male", "Female", "Other", "Prefer not to say"],
                  value="Male",
                  info="Biological sex for health calculations"
              )

              age = gr.Number(
                  label="🎂 Age",
                  value=25,
                  minimum=13,
                  maximum=120,
                  step=1,
                  info="Age in years"
              )

              weight = gr.Number(
                  label="⚖️ Weight (kg)",
                  value=70.0,
                  minimum=30.0,
                  maximum=300.0,
                  step=0.1,
                  info="Current weight in kilograms"
              )

              height = gr.Number(
                  label="📏 Height (cm)",
                  value=175.0,
                  minimum=100.0,
                  maximum=250.0,
                  step=0.1,
                  info="Height in centimeters"
              )

              # Preferences Section
              gr.Markdown("### 🍽️ Meal Preferences & Dietary Restrictions")
              with gr.Row():
                  diet_type = gr.Dropdown(
                      label="🥗 Diet Type",
                      choices=[
                          "No Preference", "Vegetarian", "Vegan", "Pescatarian",
                          "Keto", "Paleo", "Mediterranean", "Low Carb", "Gluten Free", "Dairy Free"
                      ],
                      value="No Preference",
                      info="Select your primary dietary preference"
                  )

              allergies = gr.CheckboxGroup(
                  label="⚠️ Food Allergies & Intolerances",
                  choices=[
                      "Nuts (Tree nuts)", "Peanuts", "Shellfish", "Fish",
                      "Milk/Dairy", "Eggs", "Soy", "Wheat/Gluten",
                      "Sesame", "Lactose Intolerant", "Other"
                  ],
                  info="⚠️ IMPORTANT: Select all allergies and intolerances"
              )

              other_allergies = gr.Textbox(
                  label="🚨 Other Allergies/Restrictions",
                  placeholder="e.g., tomatoes, specific medications, religious restrictions...",
                  info="Specify any other dietary restrictions or allergies"
              )

              # Activity Section
              gr.Markdown("### 🏃‍♂️ Physical Activity")
              total_steps = gr.Number(label="Total Steps (Week)", value=65000, info="Your current target will be shown")
              zone_minutes = gr.Number(label="Zone Minutes (Week)", value=120, info="Your current target will be shown")
              exercise_sessions = gr.Number(label="Exercise Sessions", value=3, info="Your current target will be shown")

              # Sleep & Mood Section
              gr.Markdown("### 😴 Sleep & Mood")
              sleep_hours = gr.Textbox(
                  label="Sleep Hours (7 days)",
                  value="7.5,6.8,8.2,7.0,6.5,8.5,7.8",
                  info="Enter daily sleep hours separated by commas"
              )
              mood_scores = gr.Textbox(
                  label="Mood Scores (7 days)",
                  value="4,3.5,5,3,4,2.5,5",
                  info="Rate your daily mood from 1-5, separated by commas"
              )

              # Food Section
              gr.Markdown("### 🍎 Weekly Nutrition Intake")
              gr.Markdown("*Enter your total consumption for the entire week*")

              with gr.Row():
                  dairy = gr.Number(label="🥛 Dairy (liters)", value=3.5, info="Milk, yogurt, cheese")
                  water_liters = gr.Number(label="💧 Water (liters)", value=48, info="Your current target will be shown")

              with gr.Row():
                  fruits = gr.Number(label="🍎 Fruits (grams)", value=2100, info="Your current target will be shown")
                  vegetables = gr.Number(label="🥬 Vegetables (grams)", value=3500, info="Your current target will be shown")

              with gr.Row():
                  legumes = gr.Number(label="🫘 Legumes (grams)", value=350, info="Beans, lentils, peas")
                  nuts_seeds = gr.Number(label="🥜 Nuts/Seeds (grams)", value=210, info="Almonds, walnuts, chia seeds")

              with gr.Row():
                  meat = gr.Number(label="🍗 Meat (grams)", value=700, info="Chicken, beef, fish")
                  grains = gr.Number(label="🌾 Grains (grams)", value=2800, info="Rice, bread, pasta, oats")

              # Action buttons
              with gr.Row():
                  analyze_btn = gr.Button("🤖 Get AI Recommendations", variant="primary", size="lg")
                  load_targets_btn = gr.Button("📊 Load My Targets", variant="secondary")

          with gr.Column():
              main_output = gr.Markdown("""
  ## 📋 How to Use

  1. **Enter your User ID** - This tracks your progress over time
  2. **Set your personal details** - Age, sex, weight, height for personalized calculations
  3. **Set your dietary preferences** - Diet type, allergies, meal preferences
  4. **⚠️ CRITICAL: Select all allergies** - Ensures safe recommendations
  5. **Fill in your weekly data** - Activity, sleep, mood, nutrition
  6. **Click 'Get AI Recommendations'** - Receive personalized advice

  ### 🎯 Default Weekly Targets:
  - **Steps:** 70,000+ (10,000 daily)
  - **Zone Minutes:** 150+ (cardio exercise)
  - **Sleep:** 8 hours nightly
  - **Water:** 14+ liters (2L daily)
  - **Fruits:** 2100+ grams
  - **Vegetables:** 3500+ grams
  - **Exercise Sessions:** 4+

  *Go to 'Customize Targets' tab to modify these targets for your personal goals.*

  ### 🔒 Privacy & Safety:
  - All data stored locally on your device
  - AI recommendations respect your dietary restrictions
  - Always consult healthcare providers for medical advice
  """)

      # FIXED: Store components with correct order including demographics
      self.main_components = {
          'user_id': user_id,
          'week_start': week_start,
          'sex': sex,           # ADDED: Missing demographic fields
          'age': age,           # ADDED
          'weight': weight,     # ADDED
          'height': height,     # ADDED
          'total_steps': total_steps,
          'zone_minutes': zone_minutes,
          'exercise_sessions': exercise_sessions,
          'sleep_hours': sleep_hours,
          'mood_scores': mood_scores,
          'dairy_liters': dairy,
          'legumes_grams': legumes,
          'meat_grams': meat,
          'fruits_grams': fruits,
          'vegetables_grams': vegetables,
          'grains_grams': grains,
          'nuts_seeds_grams': nuts_seeds,
          'water_liters': water_liters,
          'diet_type': diet_type,
          'allergies': allergies,
          'other_allergies': other_allergies,
          'output': main_output
      }

      # FIXED: Connect button with correct inputs order
      analyze_btn.click(
          fn=self.analyze_wellbeing_with_preferences,
          inputs=[
              user_id, week_start, sex, age, weight, height,  # Demographics in correct order
              total_steps, zone_minutes, exercise_sessions,
              sleep_hours, mood_scores, dairy, legumes, meat,
              fruits, vegetables, grains, nuts_seeds, water_liters,
              diet_type, allergies, other_allergies
          ],
          outputs=[main_output]
      )

      load_targets_btn.click(
          fn=self.load_user_targets,
          inputs=[user_id],
          outputs=[main_output]
      )

    def analyze_wellbeing_with_preferences(self, user_id, week_start, sex, age, weight, height,
                                     total_steps, zone_minutes, exercise_sessions,
                                     sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
                                     fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
                                     diet_type, allergies, other_allergies):
      """Enhanced analysis with agent support, custom targets, and user demographics"""

      try:
          # Input validation
          if not user_id or not user_id.strip():
              return "❌ Error: Please enter a valid User ID"

          if not week_start:
              return "❌ Error: Please enter a week start date"

          # Validate demographic data
          if age is not None and (age < 13 or age > 120):
              return "❌ Error: Please enter a valid age (13-120 years)"

          if weight is not None and (weight < 30 or weight > 300):
              return "❌ Error: Please enter a valid weight (30-300 kg)"

          if height is not None and (height < 100 or height > 250):
              return "❌ Error: Please enter a valid height (100-250 cm)"

          # Load user's custom targets
          user_targets = self.get_user_targets(user_id.strip())

          # Calculate BMI if weight and height are provided
          bmi = None
          if weight and height:
              height_m = height / 100  # Convert cm to meters
              bmi = round(weight / (height_m ** 2), 1)

          # Prepare enhanced data with demographics
          week_data = {
              'user_id': user_id.strip(),
              'week_start': week_start,
              'demographics': {
                  'sex': sex,
                  'age': int(age) if age else None,
                  'weight': float(weight) if weight else None,
                  'height': float(height) if height else None,
                  'bmi': bmi
              },
              'total_steps': max(0, int(total_steps)) if total_steps else 0,
              'zone_minutes': max(0, int(zone_minutes)) if zone_minutes else 0,
              'exercise_sessions': max(0, int(exercise_sessions)) if exercise_sessions else 0,
              'sleep_hours': sleep_hours,
              'mood_scores': mood_scores,
              'dairy_liters': max(0, float(dairy_liters)) if dairy_liters else 0,
              'legumes_grams': max(0, float(legumes_grams)) if legumes_grams else 0,
              'meat_grams': max(0, float(meat_grams)) if meat_grams else 0,
              'fruits_grams': max(0, float(fruits_grams)) if fruits_grams else 0,
              'vegetables_grams': max(0, float(vegetables_grams)) if vegetables_grams else 0,
              'grains_grams': max(0, float(grains_grams)) if grains_grams else 0,
              'nuts_seeds_grams': max(0, float(nuts_seeds_grams)) if nuts_seeds_grams else 0,
              'water_liters': max(0, float(water_liters)) if water_liters else 0,
              'preferences': {
                  'diet_type': diet_type,
                  'allergies': allergies if allergies else [],
                  'other_allergies': other_allergies.strip() if other_allergies else ""
              },
              'targets': user_targets
          }

          recommendation = self.wellbeing_system.analyze_and_recommend(user_id.strip(), week_data)

          # Enhanced output with custom targets and demographics
          output = self._format_analysis_output_with_targets(week_data, recommendation, user_targets)

          return output

      except Exception as e:
          print(f"❌ Analysis error: {str(e)}")
          import traceback
          traceback.print_exc()
          return f"❌ Error: {str(e)}\n\nPlease check your input format and try again."


    def _format_food_data(self, food_data: Dict) -> str:
      """Format food data with proper units"""
      if not food_data:
          return "• No food data provided"

      food_lines = []
      unit_mapping = {
          'dairy_liters': ('Dairy', 'L'),
          'water_liters': ('Water', 'L'),
          'legumes_grams': ('Legumes', 'g'),
          'meat_grams': ('Meat', 'g'),
          'fruits_grams': ('Fruits', 'g'),
          'vegetables_grams': ('Vegetables', 'g'),
          'grains_grams': ('Grains', 'g'),
          'nuts_seeds_grams': ('Nuts/Seeds', 'g')
      }

      for key, amount in food_data.items():
          if amount > 0 and key in unit_mapping:
              name, unit = unit_mapping[key]
              food_lines.append(f"• {name}: {amount:.0f}{unit}")
          elif amount > 0:
              clean_name = key.replace('_', ' ').title()
              food_lines.append(f"• {clean_name}: {amount}")

      return '\n'.join(food_lines) if food_lines else "• No significant food intake recorded"

    def _build_progress_context(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Build progress context from user history"""
        if len(user_history) < 2:
            return "PROGRESS CONTEXT:\n• First week or limited history available"

        prev_week = user_history[-2]

        step_change = current_week.get('total_steps', 0) - prev_week.get('total_steps', 0)
        zone_change = current_week.get('zone_minutes', 0) - prev_week.get('zone_minutes', 0)
        sleep_change = current_week.get('avg_sleep', 0) - prev_week.get('avg_sleep', 0)

        context = f"""PROGRESS FROM LAST WEEK:
- Steps: {step_change:+,} change
- Zone minutes: {zone_change:+} change
- Sleep: {sleep_change:+.1f} hours change
- Total weeks tracked: {len(user_history)}"""

        return context

    def _build_preferences_context(self, preferences: Dict) -> str:
        """Build dietary preferences context for the LLM"""
        if not preferences:
            return "DIETARY PREFERENCES:\n• No specific preferences provided"

        context_lines = ["DIETARY PREFERENCES & RESTRICTIONS:"]

        diet_type = preferences.get('diet_type', 'No Preference')
        if diet_type != 'No Preference':
            context_lines.append(f"• Diet Type: {diet_type}")

        allergies = preferences.get('allergies', [])
        other_allergies = preferences.get('other_allergies', '')

        if allergies or other_allergies:
            context_lines.append("• ⚠️ ALLERGIES/RESTRICTIONS (MUST AVOID):")
            for allergy in allergies:
                context_lines.append(f"  - {allergy}")
            if other_allergies:
                context_lines.append(f"  - {other_allergies}")




        return '\n'.join(context_lines)
    def _create_targets_tab(self):
        """Create the targets customization tab"""

        gr.Markdown("## 🎯 Customize Your Personal Health Targets")
        gr.Markdown("### Set personalized goals based on your fitness level, health conditions, and personal objectives")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                target_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter the user ID to customize targets for")

                gr.Markdown("### 🏃‍♂️ Activity Targets")
                target_weekly_steps = gr.Number(
                    label="Weekly Steps Target",
                    value=70000,
                    info="Default: 70,000 (10,000 daily). Adjust based on fitness level."
                )
                target_zone_minutes = gr.Number(
                    label="Weekly Zone Minutes Target",
                    value=150,
                    info="Default: 150 minutes. WHO recommends 150+ minutes moderate exercise."
                )
                target_exercise_sessions = gr.Number(
                    label="Weekly Exercise Sessions Target",
                    value=4,
                    info="Default: 4 sessions. Include both cardio and strength training."
                )

                gr.Markdown("### 😴 Sleep & Recovery Targets")
                target_daily_sleep = gr.Number(
                    label="Daily Sleep Hours Target",
                    value=8.0,
                    info="Default: 8 hours. Range: 7-9 hours for most adults."
                )

                with gr.Row():
                    save_targets_btn = gr.Button("💾 Save My Targets", variant="primary", size="lg")
                    load_current_targets_btn = gr.Button("📊 Load Current Targets", variant="secondary")
                    reset_defaults_btn = gr.Button("🔄 Reset to Defaults", variant="secondary")


        # Store target components
        self.target_components = {
            'user_id': target_user_id,
            'weekly_steps': target_weekly_steps,
            'zone_minutes': target_zone_minutes,
            'exercise_sessions': target_exercise_sessions,
            'daily_sleep': target_daily_sleep,
        }

        # Connect buttons
        save_targets_btn.click(
            fn=self.save_user_targets,
            inputs=list(self.target_components.values())[:-1],
            # outputs=[targets_output]
        )

        load_current_targets_btn.click(
            fn=self.load_user_targets_display,
            inputs=[target_user_id],
            outputs=list(self.target_components.values()) # Update all target number boxes and output
        )

        reset_defaults_btn.click(
            fn=self.reset_to_defaults,
            outputs=list(self.target_components.values())
        )


    def _create_history_tab(self):
        """Create the history review tab"""

        gr.Markdown("## 📈 Progress History & Analytics")
        gr.Markdown("### Track your health journey and see improvements over time")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                history_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter user ID to view history")

                gr.Markdown("### 📊 Analysis Options")
                analysis_type = gr.Radio(
                    label="📈 View Type",
                    choices=[
                        "📊 Complete Progress Summary",
                        "📈 Weekly Trends Analysis",
                        "🎯 Target Achievement Report",
                        "📋 Raw Data Export",
                        "🏆 Achievement Milestones"
                    ],
                    value="📊 Complete Progress Summary",
                    info="Choose what type of analysis you want to see"
                )

                weeks_to_show = gr.Slider(
                    label="📅 Weeks to Include",
                    minimum=1,
                    maximum=12,
                    value=4,
                    step=1,
                    info="How many recent weeks to analyze"
                )

                with gr.Row():
                    view_history_btn = gr.Button("📊 View Progress", variant="primary", size="lg")
                    export_data_btn = gr.Button("💾 Export Data", variant="secondary")
                    delete_history_btn = gr.Button("🗑️ Clear History", variant="stop")



        # Store history components
        self.history_components = {
            'user_id': history_user_id,
            'analysis_type': analysis_type,
            'weeks_to_show': weeks_to_show,
            # 'output': history_output
        }

    def _format_preferences_display(self, preferences: dict) -> str:
        """Format preferences for nice display"""

        lines = ["### 🍽️ Your Dietary Profile:"]

        # Diet type
        if preferences.get('diet_type', 'No Preference') != "No Preference":
            lines.append(f"- **Diet:** {preferences['diet_type']}")

        # Allergies (most important!)
        if preferences.get('allergies'):
            allergy_list = ", ".join(preferences['allergies'])
            lines.append(f"- **⚠️ ALLERGIES:** {allergy_list}")

        if preferences.get('other_allergies'):
            lines.append(f"- **⚠️ OTHER RESTRICTIONS:** {preferences['other_allergies']}")

        if len(lines) == 1:  # Only the header
            lines.append("- No specific dietary preferences set")

        return "\n".join(lines)



    def _format_analysis_output_with_targets(self, week_data, recommendation, user_targets):
      """Format analysis output including custom targets"""

      daily_steps = week_data['total_steps'] / 7
      daily_water = week_data['water_liters'] / 7

      # Safely get avg_sleep and avg_mood with defaults
      avg_sleep = week_data.get('avg_sleep', 0)
      avg_mood = week_data.get('avg_mood', 0)

      # Compare against custom targets
      targets_comparison = f"""
  ### 🎯 Target Performance:
  - **Steps:** {week_data['total_steps']:,} / {user_targets['weekly_steps']:,} {'✅' if week_data['total_steps'] >= user_targets['weekly_steps'] else '⚠️'} ({((week_data['total_steps'] / user_targets['weekly_steps']) * 100):.0f}%)
  - **Zone Minutes:** {week_data['zone_minutes']} / {user_targets['weekly_zone_minutes']} {'✅' if week_data['zone_minutes'] >= user_targets['weekly_zone_minutes'] else '⚠️'} ({((week_data['zone_minutes'] / user_targets['weekly_zone_minutes']) * 100):.0f}%)
  - **Sleep:** {avg_sleep:.1f}h / {user_targets['daily_sleep_hours']}h {'✅' if avg_sleep >= user_targets['daily_sleep_hours'] else '⚠️'}
  - **Water:** {week_data['water_liters']:.1f}L / {user_targets['weekly_water_liters']}L {'✅' if week_data['water_liters'] >= user_targets['weekly_water_liters'] else '⚠️'}
"""

      preferences_display = self._format_preferences_display(week_data['preferences'])

      return f"""# 🏥 Personalized Wellbeing Analysis & Recommendations

  ## 👤 User: {week_data['user_id']} | 📅 Week: {week_data['week_start']}

  {preferences_display}

  {targets_comparison}

  ### 📊 Your Weekly Data Summary:
  **🏃‍♂️ Activity:**
  - **Steps:** {week_data['total_steps']:,} ({daily_steps:.0f}/day)
  - **Zone Minutes:** {week_data['zone_minutes']}
  - **Exercise Sessions:** {week_data['exercise_sessions']}

  **😴 Sleep & Mood:**
  - **Sleep:** {week_data['sleep_hours']} hours daily
  - **Mood:** {week_data['mood_scores']}/10 daily scores

  **🍎 Nutrition:**
  - 🥛 **Dairy:** {week_data['dairy_liters']:.1f}L | 💧 **Water:** {week_data['water_liters']:.1f}L ({daily_water:.1f}L/day)
  - 🍎 **Fruits:** {week_data['fruits_grams']:.0f}g | 🥬 **Vegetables:** {week_data['vegetables_grams']:.0f}g
  - 🫘 **Legumes:** {week_data['legumes_grams']:.0f}g | 🍗 **Meat:** {week_data['meat_grams']:.0f}g
  - 🌾 **Grains:** {week_data['grains_grams']:.0f}g | 🥜 **Nuts/Seeds:** {week_data['nuts_seeds_grams']:.0f}g

  ---

  ## 🤖 AI Recommendations
  *Personalized based on your data, preferences, allergies, and custom targets*

  {recommendation}
  """

    def load_user_targets(self, user_id):
        """Load user targets and format for display in main tab"""
        try:
            if not user_id or not user_id.strip():
                return "❌ Please enter a valid User ID"

            targets = self.get_user_targets(user_id.strip())

            return f"""# 📊 Your Current Targets Loaded

## 🎯 Targets for {user_id}:

### 🏃‍♂️ Activity Targets:
- **Weekly Steps:** {targets['weekly_steps']:,}
- **Zone Minutes:** {targets['weekly_zone_minutes']}
- **Exercise Sessions:** {targets['weekly_exercise_sessions']}

### 😴 Recovery Target:
- **Daily Sleep:** {targets['daily_sleep_hours']} hours

### 🍎 Nutrition Targets:
- **Weekly Water:** {targets['weekly_water_liters']} liters
- **Weekly Fruits:** {targets['weekly_fruits_grams']:,} grams
- **Weekly Vegetables:** {targets['weekly_vegetables_grams']:,} grams

*These targets will be used to evaluate your progress when you get AI recommendations.*

**💡 Tip:** Go to the 'Customize Targets' tab to modify these values if needed.
"""

        except Exception as e:
            return f"❌ Error loading targets: {str(e)}"

    def save_user_targets(self, user_id, weekly_steps, zone_minutes, exercise_sessions,
                          daily_sleep, weekly_water):
        """Saves user's custom targets to a file"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to save targets."

        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        try:
            targets_data = {
                'weekly_steps': int(weekly_steps) if weekly_steps else self.default_targets['weekly_steps'],
                'weekly_zone_minutes': int(zone_minutes) if zone_minutes else self.default_targets['weekly_zone_minutes'],
                'weekly_exercise_sessions': int(exercise_sessions) if exercise_sessions else self.default_targets['weekly_exercise_sessions'],
                'daily_sleep_hours': float(daily_sleep) if daily_sleep else self.default_targets['daily_sleep_hours'],
                'weekly_water_liters': float(weekly_water) if weekly_water else self.default_targets['weekly_water_liters'],
              }

            with open(targets_file, 'w') as f:
                json.dump(targets_data, f, indent=2)

            return f"✅ Targets saved successfully for user '{user_id.strip()}'."

        except Exception as e:
            return f"❌ Error saving targets: {str(e)}"

    def get_user_targets(self, user_id: str) -> Dict:
        """Loads user's custom targets, or returns defaults"""
        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        if os.path.exists(targets_file):
            try:
                with open(targets_file, 'r') as f:
                    targets = json.load(f)
                    # Ensure all keys from default are present (for backward compatibility)
                    full_targets = self.default_targets.copy()
                    full_targets.update(targets)
                    return full_targets
            except Exception as e:
                print(f"⚠️ Warning: Could not load targets for user '{user_id.strip()}': {str(e)}. Using defaults.")
                return self.default_targets
        else:
            return self.default_targets

    def load_user_targets_display(self, user_id):
        """Loads user targets and formats for display in the targets tab"""
        if not user_id or not user_id.strip():
             return "❌ Error: Please enter a valid User ID to load targets.", *[gr.Number(value=self.default_targets[key]) for key in self.default_targets] # Return defaults to update UI

        targets = self.get_user_targets(user_id.strip())

        output_message = f"✅ Loaded targets for user '{user_id.strip()}'."

        # Return the message and the target values to update the Gradio Number components
        return (
            output_message,
            targets.get('weekly_steps', self.default_targets['weekly_steps']),
            targets.get('weekly_zone_minutes', self.default_targets['weekly_zone_minutes']),
            targets.get('weekly_exercise_sessions', self.default_targets['weekly_exercise_sessions']),
            targets.get('daily_sleep_hours', self.default_targets['daily_sleep_hours']),
            targets.get('weekly_water_liters', self.default_targets['weekly_water_liters']),
            )

    def reset_to_defaults(self):
        """Resets the target input fields to default values"""
        return (
            "🎯 Resetting targets to defaults.",
            self.default_targets['weekly_steps'],
            self.default_targets['weekly_zone_minutes'],
            self.default_targets['weekly_exercise_sessions'],
            self.default_targets['daily_sleep_hours'],
            self.default_targets['weekly_water_liters'],
        )

    def view_user_history(self, user_id, analysis_type, weeks_to_show):
        """Displays user history based on selected analysis type"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to view history."

        history = self.wellbeing_system.data_manager.get_user_history(user_id.strip())

        if not history:
            return f"📊 No history found for user '{user_id.strip()}'."

        # Limit history to weeks_to_show
        recent_history = history[-weeks_to_show:]

        output = f"## 📈 History for User: {user_id.strip()}\n"
        output += f"📊 **Analysis Type:** {analysis_type}\n"
        output += f"📅 **Weeks Included:** {len(recent_history)}\n\n"

        if analysis_type == "📊 Complete Progress Summary":
            output += "### Weekly Summary (Most Recent First)\n"
            for week in reversed(recent_history):
                output += f"#### Week Start: {week.get('week_start', 'Unknown')}\n"
                output += f"- Steps: {week.get('total_steps', 0):,}\n"
                output += f"- Zone Minutes: {week.get('zone_minutes', 0)}\n"
                output += f"- Sleep (Avg): {week.get('avg_sleep', 0):.1f}h\n"
                output += f"- Mood (Avg): {week.get('avg_mood', 0):.1f}/10\n"
                output += f"- Exercise Sessions: {week.get('exercise_sessions', 0)}\n"
                # Add a snippet of the recommendation if available
                output += f"- Recommendation Snippet: {week.get('recommendation', 'N/A')[:100]}...\n\n"

        elif analysis_type == "📈 Weekly Trends Analysis":
             # This would ideally involve plotting, but we'll provide text summary
            progress = self.wellbeing_system.get_user_progress(user_id.strip())
            output += "### Trends (Last 2 Weeks vs Before)\n"
            if 'message' in progress:
                 output += progress['message']
            else:
                output += f"- Steps Change (Last Week vs Previous): {progress.get('steps_change', 0):+,}\n"
                output += f"- Zone Minutes Change: {progress.get('zone_change', 0):+,}\n"
                output += f"- Sleep Change: {progress.get('sleep_change', 0):+.1f}h\n"
                output += f"- Mood Change: {progress.get('mood_change', 0):+.1f}/10\n"
                if 'trends' in progress:
                    output += "\n### Longer Term Trends (Last 4 Weeks Avg)\n"
                    output += f"- Avg Steps: {progress['trends'].get('avg_steps_4weeks', 0):,.0f}\n"
                    output += f"- Avg Sleep: {progress['trends'].get('avg_sleep_4weeks', 0):.1f}h\n"
                    output += f"- Consistency Score (Steps): {progress['trends'].get('consistency_score', 0):.1f}/10\n"


        elif analysis_type == "🎯 Target Achievement Report":
            user_targets = self.get_user_targets(user_id.strip())
            output += "### Target Achievement (Based on Latest Week)\n"
            latest_week = recent_history[-1] if recent_history else {}

            if latest_week:
                 output += f"- Weekly Steps: {latest_week.get('total_steps', 0):,} / {user_targets['weekly_steps']:,} ({((latest_week.get('total_steps', 0) / user_targets['weekly_steps']) * 100):.0f}%) {'✅' if latest_week.get('total_steps', 0) >= user_targets['weekly_steps'] else '⚠️'}\n"
                 output += f"- Weekly Zone Minutes: {latest_week.get('zone_minutes', 0)} / {user_targets['weekly_zone_minutes']} ({((latest_week.get('zone_minutes', 0) / user_targets['weekly_zone_minutes']) * 100):.0f}%) {'✅' if latest_week.get('zone_minutes', 0) >= user_targets['weekly_zone_minutes'] else '⚠️'}\n"
                 output += f"- Daily Sleep (Avg): {latest_week.get('avg_sleep', 0):.1f}h / {user_targets['daily_sleep_hours']}h {'✅' if latest_week.get('avg_sleep', 0) >= user_targets['daily_sleep_hours'] else '⚠️'}\n"
                 output += f"- Weekly Water (Avg): {latest_week.get('water_liters', 0):.1f}L / {user_targets['weekly_water_liters']}L {'✅' if latest_week.get('water_liters', 0) >= user_targets['weekly_water_liters'] else '⚠️'}\n" # Assuming water is stored with units in processed data
                 output += f"- Weekly Fruits (Avg): {latest_week.get('fruits_grams', 0):.0f}g / {user_targets['weekly_fruits_grams']:.0f}g {'✅' if latest_week.get('fruits_grams', 0) >= user_targets['weekly_fruits_grams'] else '⚠️'}\n"
                 output += f"- Weekly Vegetables (Avg): {latest_week.get('vegetables_grams', 0):.0f}g / {user_targets['weekly_vegetables_grams']:.0f}g {'✅' if latest_week.get('vegetables_grams', 0) >= user_targets['weekly_vegetables_grams'] else '⚠️'}\n"
                 output += f"- Weekly Exercise Sessions: {latest_week.get('exercise_sessions', 0)} / {user_targets['weekly_exercise_sessions']} {'✅' if latest_week.get('exercise_sessions', 0) >= user_targets['weekly_exercise_sessions'] else '⚠️'}\n"

            else:
                output += "No recent data to compare against targets."


        elif analysis_type == "📋 Raw Data Export":
            output += "### Raw Data (JSON Format)\n"
            output += "```json\n"
            output += json.dumps(recent_history, indent=2)
            output += "\n```\n"

        elif analysis_type == "🏆 Achievement Milestones":
            output += "### Achievements (Conceptual - requires more logic)\n"
            output += "This feature is under development. Here are some potential milestones:\n"
            output += "- First week hitting all targets!\n"
            output += "- Longest streak of 10k+ steps daily.\n"
            output += "- Personal best for zone minutes.\n"
            output += "- Consistent 8 hours of sleep for a month.\n"


        return output

    def export_user_data(self, user_id):
        """Exports all user data to a JSON file"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to export data."

        history = self.wellbeing_system.data_manager.get_user_history(user_id.strip())
        recommendations_dir = f"{self.wellbeing_system.data_manager.data_dir}/recommendations"
        user_rec_files = [f for f in os.listdir(recommendations_dir) if f.startswith(f"{user_id.strip()}_") and f.endswith('.json')]

        all_data = {"history": history, "recommendations": []}

        for rec_file in user_rec_files:
            try:
                with open(f"{recommendations_dir}/{rec_file}", 'r') as f:
                    all_data["recommendations"].append(json.load(f))
            except Exception as e:
                print(f"⚠️ Warning: Could not load recommendation file {rec_file}: {str(e)}")


        export_filename = f"wellbeing_export_{user_id.strip()}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        export_filepath = f"{self.wellbeing_system.data_manager.data_dir}/{export_filename}"

        try:
            with open(export_filepath, 'w') as f:
                json.dump(all_data, f, indent=2)
            return f"✅ Data exported successfully to `{export_filepath}`."
        except Exception as e:
            return f"❌ Error exporting data: {str(e)}"

    def delete_user_history(self, user_id):
        """Deletes all history and recommendations for a user"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to delete history."

        user_file = f"{self.wellbeing_system.data_manager.data_dir}/users/{user_id.strip()}.json"
        recommendations_dir = f"{self.wellbeing_system.data_manager.data_dir}/recommendations"

        deleted_count = 0
        try:
            # Delete user history file
            if os.path.exists(user_file):
                os.remove(user_file)
                deleted_count += 1

            # Delete recommendation files
            user_rec_files = [f for f in os.listdir(recommendations_dir) if f.startswith(f"{user_id.strip()}_") and f.endswith('.json')]
            for rec_file in user_rec_files:
                try:
                    os.remove(f"{recommendations_dir}/{rec_file}")
                    deleted_count += 1
                except Exception as e:
                    print(f"⚠️ Warning: Could not delete recommendation file {rec_file}: {str(e)}")

            if deleted_count > 0:
                 return f"🗑️ Successfully deleted {deleted_count} files for user '{user_id.strip()}'."
            else:
                 return f"ℹ️ No data found for user '{user_id.strip()}'. Nothing to delete."

        except Exception as e:
            return f"❌ Error deleting history: {str(e)}"


    def comprehensive_system_test(self):
        """Runs a comprehensive test of the system components"""
        output = "## 🧪 Comprehensive System Test Results\n"

        try:
            # Test LLM loading and generation
            output += "### 🤖 LLM Test\n"
            llm_test_result = self.wellbeing_system.llm.test_model()
            output += f"- Status: {llm_test_result}\n\n"

            # Test Data Management (basic save/load cycle)
            output += "### 💾 Data Management Test\n"
            test_user = "test_user_system"
            test_week = {
                'week_start': '2024-07-01',
                'total_steps': 75000,
                'zone_minutes': 160,
                'exercise_sessions': 4,
                'sleep_hours': '8,7.5,8,7,7.5,8,8',
                'mood_scores': '8,7,9,8,7,9,8',
                'dairy_liters': 3.0,
                'water_liters': 15.0,
                'fruits_grams': 2200,
                'vegetables_grams': 3600
            }
            test_rec = "Test recommendation output."

            try:
                self.wellbeing_system.data_manager.save_user_week(test_user, test_week)
                history = self.wellbeing_system.data_manager.get_user_history(test_user)
                output += f"- Save/Load History: {'✅ Success' if len(history) > 0 and history[-1].get('total_steps') == 75000 else '❌ Failed'}\n"

                self.wellbeing_system.data_manager.save_recommendation(test_user, test_week['week_start'], test_rec)
                last_rec = self.wellbeing_system.data_manager.get_last_recommendation(test_user)
                output += f"- Save/Load Recommendation: {'✅ Success' if last_rec == test_rec else '❌ Failed'}\n"

                # Clean up test data
                self.wellbeing_system.data_manager.delete_user_history(test_user) # This is now a method on data_manager
                cleanup_check = self.wellbeing_system.data_manager.get_user_history(test_user)
                output += f"- Cleanup: {'✅ Success' if not cleanup_check else '❌ Failed'}\n"

            except Exception as e:
                output += f"- Data Management Test Failed: {str(e)}\n"
                import traceback
                output += f"```\n{traceback.format_exc()}\n```\n"


            # Test Integration (analyze_and_recommend)
            output += "### 🔗 Integration Test\n"
            try:
                # Use sample data that includes preferences
                integration_test_data = {
                    'user_id': 'integration_user',
                    'week_start': '2024-07-08',
                    'total_steps': 50000,
                    'zone_minutes': 100,
                    'exercise_sessions': 2,
                    'sleep_hours': '6.5,7,6,7.5,7,6.8,7.2',
                    'mood_scores': '6,7,6,7,6,7,6',
                    'dairy_liters': 0.5, # Low dairy
                    'water_liters': 10.0, # Low water
                    'fruits_grams': 1500,
                    'vegetables_grams': 2500,
                    'grains_grams': 2000,
                    'nuts_seeds_grams': 150,
                    'meat_grams': 500,
                    'preferences': {
                         'diet_type': 'Vegetarian',
                         'allergies': ['Nuts (Tree nuts)'],
                         'other_allergies': 'Lactose Intolerant'
                    }
                }
                # Use default targets for this test
                integration_test_data['targets'] = self.default_targets.copy()


                integration_recommendation = self.wellbeing_system.analyze_and_recommend('integration_user', integration_test_data)

                output += f"- Analyze & Recommend: {'✅ Success' if len(integration_recommendation) > 100 and 'Vegetarian' in integration_recommendation and 'Nuts' not in integration_recommendation else '❌ Failed'}\n"
                output += f"  - Recommendation Length: {len(integration_recommendation)}\n"
                output += f"  - Recommendation Preview: {integration_recommendation[:150]}...\n"

                # Clean up integration test data
                self.wellbeing_system.data_manager.delete_user_history('integration_user')
                cleanup_check_int = self.wellbeing_system.data_manager.get_user_history('integration_user')
                output += f"- Cleanup: {'✅ Success' if not cleanup_check_int else '❌ Failed'}\n"


            except Exception as e:
                output += f"- Integration Test Failed: {str(e)}\n"
                import traceback
                output += f"```\n{traceback.format_exc()}\n```\n"


            output += "\n**🧪 System Test Complete.** Check the results above."

        except Exception as e:
            output += f"\n❌ An error occurred during the system test: {str(e)}"

        return output


    def check_llm_status(self):
        """Checks the status of the loaded LLM model"""
        if self.wellbeing_system.llm.model and self.wellbeing_system.llm.tokenizer:
            status = f"✅ LLM is loaded and ready.\n"
            status += f"- Base Model: {self.wellbeing_system.llm.model.config._name_or_path}\n"
            status += f"- Adapter: {self.wellbeing_system.llm.model.peft_config['default'].peft_adapter_id}\n"
            status += f"- Device: {self.wellbeing_system.llm.device}\n"
            try:
                 test_output = self.wellbeing_system.llm.test_model()
                 status += f"- Basic Test: {test_output}\n"
            except Exception as e:
                 status += f"- Basic Test Failed: {str(e)}\n"

        elif self.wellbeing_system.llm.model:
             status = "⚠️ LLM model is loaded, but tokenizer is missing.\n"
        elif self.wellbeing_system.llm.tokenizer:
             status = "⚠️ LLM tokenizer is loaded, but model is missing.\n"
        else:
            status = "❌ LLM model and tokenizer failed to load.\n"
            status += "Please check the model loading logs in the console for errors."
            status += "\n\n⚠️ **LLM Loading Issue:** The model might be too large for the available GPU RAM."
            status += "Consider trying a smaller model or a runtime with more GPU memory (e.g., A100)."


        return f"## 🤖 LLM System Status\n{status}"

    def check_storage_status(self):
        """Checks the status of the data storage directory"""
        data_dir = self.wellbeing_system.data_manager.data_dir
        output = f"## 💾 Data Storage Status\n"
        output += f"- Data Directory: `{data_dir}`\n"

        try:
            os.makedirs(data_dir, exist_ok=True)
            output += "- Directory exists or created: ✅\n"

            users_dir = f"{data_dir}/users"
            recs_dir = f"{data_dir}/recommendations"

            os.makedirs(users_dir, exist_ok=True)
            os.makedirs(recs_dir, exist_ok=True)

            output += f"- Users directory (`{users_dir}`) exists or created: ✅\n"
            output += f"- Recommendations directory (`{recs_dir}`) exists or created: ✅\n"

            # Count files
            user_files = [f for f in os.listdir(users_dir) if f.endswith('.json')]
            rec_files = [f for f in os.listdir(recs_dir) if f.endswith('.json')]

            output += f"- Number of user data files: {len(user_files)}\n"
            output += f"- Number of recommendation files: {len(rec_files)}\n"

            # Check if a dummy file can be written
            test_file = f"{data_dir}/test_write_{datetime.now().strftime('%Y%m%d%H%M%S')}.txt"
            try:
                with open(test_file, 'w') as f:
                    f.write("test")
                os.remove(test_file)
                output += "- Write permission: ✅\n"
            except Exception as e:
                output += f"- Write permission: ❌ ({str(e)})\n"

        except Exception as e:
            output += f"❌ An error occurred checking storage: {str(e)}\n"

        return output

    def analyze_with_agent_toggle(self, user_id, week_start, total_steps, zone_minutes, exercise_sessions,
                             sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
                             fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
                             diet_type, allergies, other_allergies):
      """Handle analysis with agent toggle support"""

      try:


          # Call your existing analysis method
          result = self.analyze_wellbeing_with_preferences(
              user_id, week_start, total_steps, zone_minutes, exercise_sessions,
              sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
              fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
              diet_type, allergies, other_allergies
          )

          return result

      except Exception as e:
          return f"❌ Error: {str(e)}"

# Main execution

In [ ]:
# Uncomment to run:
app = WellbeingApp(base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B", adapter_path="AnjaliNV/WellBeing_LLM")
demo = app.create_wellbeing_app()
demo.launch(share=True, debug=True)

# Fine Tuning

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import Dataset
import gc
import os
import random
import re
from typing import List, Dict

class CleanHealthCoachTrainer:
    """
    Clean training system that prevents number confusion from the start
    """

    def __init__(self, model_name: str):
        self.model_name = model_name
        self.clear_memory()

        print("🚀 Setting up CLEAN health coach training...")

        # Memory optimization
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

        # Quantization config
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=False,
        )

        # Load tokenizer
        print("📦 Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True,
            use_fast=True
        )

        # Load model
        print("🤖 Loading model...")
        self.base_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            use_cache=False,
            torch_dtype=torch.float16,
            device_map="auto"
        )

        # Prepare for training
        self.base_model = prepare_model_for_kbit_training(
            self.base_model,
            use_gradient_checkpointing=True
        )

        # Setup LoRA
        self.model = self.setup_lora()

        # Configure tokenizer
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        print("✅ Clean setup complete!")

    def clear_memory(self):
        """Clear GPU memory"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

    def setup_lora(self):
        """Setup LoRA for fine-tuning"""
        lora_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            inference_mode=False,
            r=32,
            lora_alpha=64,
            lora_dropout=0.1,
            target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
            bias="none",
        )

        model = get_peft_model(self.base_model, lora_config)
        model.train()
        model.print_trainable_parameters()
        return model

    def expand_natural_examples(self):
      """Create more natural language examples"""

      additional_examples = [
          {
              "input": "42-year-old male, 50,000 weekly steps, 100 zone minutes, wants weight loss",
              "output": """**1) Food Recommendation**
  a) Overall Assessment: Your current eating habits provide a good foundation for weight loss.
  b) Areas to Improve: Focus on creating a moderate calorie deficit while maintaining nutrition quality.
  c) Meal and Food Suggestions: Increase vegetables at each meal and choose lean proteins like chicken or fish.

  **2) Physical Exercise**
  a) Activity and Performance Analysis: Your daily steps are around 7,000, which is a good start.
  b) Zone Minutes and Intensity Feedback: Adding 50 more minutes of cardio weekly would boost weight loss.
  c) Strength Training Analysis: Include 2-3 strength sessions to preserve muscle during weight loss.
  d) Cardio Suggestions: Try activities you enjoy like hiking, swimming, or bike rides.

  **3) Sleep and Social Well-being**
  a) Sleep/Mood Analysis: Good sleep helps with weight loss hormones and appetite control.
  b) Suggestions: Aim for 7-8 hours nightly and consider joining fitness groups for motivation.

  **4) Overall Weekly Recommendation**
  a) Summary: Combine moderate activity increases with mindful eating for sustainable weight loss.
  b) Weekly Goals: Add 3,000 more weekly steps and include 2 strength training sessions."""
          }
      ]

      return additional_examples

    # 🚀 ADD THE NEW FUNCTION HERE:
    def create_natural_response_examples(self):
          """Generate training examples with natural, realistic language"""

          examples = [
              {
                  "input": "25-year-old male, 65,000 weekly steps, 168 zone minutes, wants health maintenance",
                  "output": """**1) Food Recommendation**
  a) Overall Assessment: Your nutrition looks well-balanced with good protein intake from meat and legumes.
  b) Areas to Improve: Consider adding more variety in fruits and vegetables for better micronutrient coverage.
  c) Meal and Food Suggestions: Try adding berries to breakfast and leafy greens to lunch for extra nutrients.

  **2) Physical Exercise**
  a) Activity and Performance Analysis: Your step count is excellent at over 9,000 daily steps.
  b) Zone Minutes and Intensity Feedback: You're exceeding the recommended 150 minutes - great work!
  c) Strength Training Analysis: Three sessions per week is perfect for maintaining fitness.
  d) Cardio Suggestions: Keep up your current routine, maybe add some variety like swimming or cycling.

  **3) Sleep and Social Well-being**
  a) Sleep/Mood Analysis: Your sleep could improve - aim for 7-8 hours consistently.
  b) Suggestions: Try a regular bedtime routine and limit screen time before bed.

  **4) Overall Weekly Recommendation**
  a) Summary: You're doing great with exercise, focus on sleep improvement.
  b) Weekly Goals: Maintain current activity level and work on getting an extra hour of sleep nightly."""
              },

              {
                  "input": "35-year-old female, 45,000 weekly steps, 90 zone minutes, wants weight loss",
                  "output": """**1) Food Recommendation**
  a) Overall Assessment: Your current intake provides good nutrition, but portion adjustments could support weight loss.
  b) Areas to Improve: Focus on increasing protein and fiber to help with satiety.
  c) Meal and Food Suggestions: Add lean proteins like chicken or fish to meals, and include more vegetables.

  **2) Physical Exercise**
  a) Activity and Performance Analysis: Your daily steps are good but could be increased gradually.
  b) Zone Minutes and Intensity Feedback: Adding 60 more minutes of cardio weekly would support your weight loss goals.
  c) Strength Training Analysis: Consider adding 2-3 strength sessions to preserve muscle during weight loss.
  d) Cardio Suggestions: Try brisk walking, cycling, or dancing for enjoyable cardio options.

  **3) Sleep and Social Well-being**
  a) Sleep/Mood Analysis: Good sleep supports weight management hormones.
  b) Suggestions: Aim for 7-8 hours nightly and consider stress management techniques.

  **4) Overall Weekly Recommendation**
  a) Summary: Gradual increases in activity plus mindful eating will support healthy weight loss.
  b) Weekly Goals: Add 3,000 more weekly steps and include 2 strength training sessions."""
              }
          ]

          return examples

    def create_clean_training_data(self):
        """Generate clean, consistent training examples with proper numbers"""

        print("📚 Creating clean training data with realistic numbers...")

        # Get your existing examples
        expanded_examples = self.expand_natural_examples()

        # 🚀 ADD NATURAL EXAMPLES:
        natural_examples = self.create_natural_response_examples()

        # Combine both types
        all_examples = natural_examples + expanded_examples

        print(f"✅ Created {len(all_examples)} training examples ({len(natural_examples)} natural)")
        return all_examples

    def generate_person_profiles(self):
        """Generate diverse, realistic person profiles"""

        profiles = []

        # Define variety of profiles
        base_profiles = [
            {"gender": "Male", "age": 28, "weight": 75, "height": 180, "activity": "moderate", "goal": "build muscle"},
            {"gender": "Female", "age": 35, "weight": 65, "height": 165, "activity": "low", "goal": "lose weight"},
            {"gender": "Male", "age": 42, "weight": 85, "height": 178, "activity": "high", "goal": "maintain fitness"},
            {"gender": "Female", "age": 29, "weight": 58, "height": 162, "activity": "moderate", "goal": "increase energy"},
            {"gender": "Male", "age": 38, "weight": 92, "height": 175, "activity": "low", "goal": "improve health"},
            {"gender": "Female", "age": 26, "weight": 70, "height": 168, "activity": "high", "goal": "athletic performance"},
        ]

        # Add variations
        diets = ["Vegetarian", "Keto", "Mediterranean", "Vegan", "No preference"]
        allergies = ["None", "Gluten", "Dairy", "Nuts", "Shellfish"]

        for base in base_profiles:
            for i in range(3):  # 3 variations each
                profile = base.copy()
                profile["diet"] = random.choice(diets)
                profile["allergies"] = random.choice(allergies)

                # Add realistic activity data based on activity level
                if profile["activity"] == "low":
                    profile["weekly_steps"] = random.randint(20000, 35000)
                    profile["zone_minutes"] = random.randint(30, 80)
                    profile["sessions"] = random.randint(0, 1)
                elif profile["activity"] == "moderate":
                    profile["weekly_steps"] = random.randint(35000, 60000)
                    profile["zone_minutes"] = random.randint(80, 150)
                    profile["sessions"] = random.randint(1, 3)
                else:  # high
                    profile["weekly_steps"] = random.randint(60000, 85000)
                    profile["zone_minutes"] = random.randint(150, 250)
                    profile["sessions"] = random.randint(3, 5)

                # Add sleep and mood data
                profile["sleep_avg"] = round(random.uniform(6.0, 8.5), 1)
                profile["mood_avg"] = random.randint(3, 8)

                profiles.append(profile)

        return profiles

    def calculate_realistic_recommendations(self, profile, metrics):
        """Calculate realistic recommendations based on person profile"""

        recommendations = {}

        # Water needs (based on weight and activity)
        base_water = profile["weight"] * 0.035  # Basic need
        if profile["activity"] == "moderate":
            base_water += 1.0
        elif profile["activity"] == "high":
            base_water += 2.0
        recommendations["water"] = round(base_water, 1)

        # Protein needs (based on weight and goal)
        if profile["goal"] == "build muscle" or profile["activity"] == "high":
            protein_ratio = 1.6
        elif profile["goal"] == "lose weight":
            protein_ratio = 1.2
        else:
            protein_ratio = 1.0
        recommendations["protein"] = round(profile["weight"] * protein_ratio)

        # Step goals
        daily_steps = profile["weekly_steps"] // 7
        if daily_steps < 7000:
            recommendations["step_target"] = daily_steps + 2000
        elif daily_steps < 10000:
            recommendations["step_target"] = 10000
        else:
            recommendations["step_target"] = daily_steps

        # Sleep recommendations
        if profile["sleep_avg"] < 7:
            recommendations["sleep_target"] = 7.5
        else:
            recommendations["sleep_target"] = profile["sleep_avg"]

        return recommendations

    def create_input_text(self, profile):
        """Create realistic input text for training"""

        daily_steps = profile["weekly_steps"] // 7

        input_text = f"""Analyze health data: {profile['gender']}, {profile['age']}, {profile['weight']}kg, {profile['height']}cm
**Activity:**
* **Steps:** {profile['weekly_steps']} weekly ({daily_steps} daily average)
* **Zone Minutes:** {profile['zone_minutes']} weekly
* **Exercise Sessions:** {profile['sessions']} weekly
**😴 Sleep & Mood:**
* **Sleep:** {profile['sleep_avg']} hours average nightly
* **Mood:** {profile['mood_avg']}/10 average
**🍎 Nutrition:**
* **Diet:** {profile['diet']} | **Allergies:** {profile['allergies']}
**Goals:** {profile['goal'].title()}"""

        return input_text

    def create_proper_output(self, profile, rec):
        """Create properly formatted output with realistic numbers"""

        daily_steps = profile["weekly_steps"] // 7

        output_text = f"""## 1) Food Recommendation
- Overall Assessment: {profile['gender']} ({profile['weight']}kg) following {profile['diet'].lower()} diet needs approximately {rec['protein']}g protein daily and {rec['water']}L water based on {profile['activity']} activity level.
- Areas to Improve: -- Target {rec['protein']}g daily protein for {profile['goal'].lower()} goals
                     -- Maintain {rec['water']}L daily water intake for optimal hydration
                     -- Consider {profile['allergies'].lower()} allergies in meal planning
- Meal and Food Suggestions: -- Proteins: Aim for {rec['protein']}g daily through {profile['diet'].lower()}-friendly sources
                             -- Hydration: Spread {rec['water']}L water throughout the day
                             -- Energy: Complex carbs to support {daily_steps} daily steps

## 2) Physical Exercise Suggestion
- Activity and Performance Analysis: Current {daily_steps} daily steps {'meets' if daily_steps >= 10000 else 'approaches' if daily_steps >= 7000 else 'falls below'} the 10,000 step recommendation.
- Zone Minutes and Intensity Feedback: {profile['zone_minutes']} weekly zone minutes {'exceeds' if profile['zone_minutes'] >= 150 else 'meets' if profile['zone_minutes'] >= 120 else 'falls below'} the 150-minute target.
- Strength Training Analysis: {profile['sessions']} weekly sessions is {'excellent' if profile['sessions'] >= 3 else 'adequate' if profile['sessions'] >= 2 else 'insufficient'} for {profile['goal'].lower()}.
- Cardio Suggestions: {'Maintain current level' if profile['zone_minutes'] >= 150 else f'Add {150 - profile["zone_minutes"]} more weekly zone minutes'} through activities suitable for {profile['activity']} individuals.

## 3) Sleep and Social Well-being
- Sleep/Mood Analysis: {profile['sleep_avg']} hours nightly sleep {'supports' if profile['sleep_avg'] >= 7 else 'may limit'} recovery for {profile['goal'].lower()} goals. Mood score of {profile['mood_avg']}/10 {'shows good' if profile['mood_avg'] >= 6 else 'indicates room for'} well-being.
- Suggestions: -- {'Maintain' if profile['sleep_avg'] >= 7.5 else 'Increase to'} {rec['sleep_target']} hours nightly for optimal recovery
               -- Engage in {profile['diet'].lower()} community groups for social support and meal ideas

## 4) Overall Weekly Recommendation
- Summary: {profile['age']}-year-old {profile['gender'].lower()} with {profile['activity']} activity level needs focus on {'maintaining excellent habits' if daily_steps >= 10000 and profile['sleep_avg'] >= 7.5 else 'improving daily movement and recovery'}.
- Weekly Goals: -- Achieve {rec['step_target']} daily steps consistently
                -- Maintain {rec['protein']}g daily protein intake
                -- Target {rec['sleep_target']} hours nightly sleep"""

        return output_text

    def create_format_instruction_prompt(self, input_text, output_text):
        """Add clear format instructions to each training example"""

        format_instruction = """You are a wellness coach AI. You MUST respond in this EXACT format with REALISTIC numbers:

CRITICAL RULES:
- Water: Use 2-10 liters per DAY (never thousands)
- Protein: Use 50-200 grams per DAY (never thousands)
- Steps: Use 1,000-20,000 per DAY (never mix with nutrition)
- Sleep: Use 5-10 hours per NIGHT
- Always use appropriate units and realistic ranges

FORMAT TEMPLATE:
## 1) Food Recommendation
- Overall Assessment: [realistic analysis with proper numbers]
- Areas to Improve: -- [specific improvement with correct units]
- Meal and Food Suggestions: -- [category]: [specific suggestions]

## 2) Physical Exercise Suggestion
- Activity and Performance Analysis: [steps analysis with daily numbers]
- Zone Minutes and Intensity Feedback: [weekly zone minutes vs 150 target]
- Strength Training Analysis: [sessions per week assessment]
- Cardio Suggestions: [specific cardio recommendations]

## 3) Sleep and Social Well-being
- Sleep/Mood Analysis: [hours per night and mood analysis]
- Suggestions: -- [sleep improvement strategies]
               -- [social well-being suggestions]

## 4) Overall Weekly Recommendation
- Summary: [comprehensive summary with key focus areas]
- Weekly Goals: -- [specific measurable goal 1]
                -- [specific measurable goal 2]
                -- [specific measurable goal 3]

REMEMBER: Use realistic numbers with proper units!

"""

        full_prompt = f"{format_instruction}Human: {input_text}\n\nAssistant: {output_text}"
        return full_prompt

    def tokenize_function(self, examples):
        """Tokenize training data"""
        tokenized = self.tokenizer(
            examples["text"],
            truncation=True,
            padding=False,
            max_length=1024,
        )
        tokenized["labels"] = tokenized["input_ids"].copy()
        return tokenized

    def train_clean_model(self, output_dir: str = "./clean_health_coach"):
        """Train the model with clean, consistent data"""

        print("🎯 Starting clean health coach training...")

        # Generate clean training data
        examples = self.create_clean_training_data()

        # Format with instructions
        training_data = []
        for example in examples:
            formatted_prompt = self.create_format_instruction_prompt(
                example["input"],
                example["output"]
            )
            training_data.append({"text": formatted_prompt})

        print(f"📊 Training on {len(training_data)} clean examples")

        # Create dataset
        dataset = Dataset.from_list(training_data)

        self.clear_memory()

        # Tokenize
        tokenized_dataset = dataset.map(
            self.tokenize_function,
            batched=True,
            batch_size=2,
            remove_columns=dataset.column_names
        )

        # Data collator
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,
            pad_to_multiple_of=8,
        )

        # Conservative training arguments for clean learning
        training_args = TrainingArguments(
            output_dir=output_dir,
            overwrite_output_dir=True,
            num_train_epochs=5,  # Conservative epochs
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=20,
            weight_decay=0.01,
            logging_steps=10,
            save_strategy="epoch",
            learning_rate=3e-4,  # Conservative learning rate
            fp16=True,
            gradient_checkpointing=True,
            remove_unused_columns=True,
            dataloader_pin_memory=False,
            dataloader_num_workers=0,
            report_to=[],
            max_grad_norm=1.0,
            save_total_limit=2,
        )

        self.clear_memory()

        # Create trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator,
        )

        print("🚀 Starting clean training (no confusion!)...")

        try:
            trainer.train()

            print("💾 Saving clean model...")
            self.model.save_pretrained(output_dir)
            self.tokenizer.save_pretrained(output_dir)

            print(f"✅ Clean health coach saved to {output_dir}")
            return self.model, self.tokenizer

        except Exception as e:
            print(f"❌ Training failed: {e}")
            return None, None

    def test_clean_model(self):
        """Test the trained model for proper responses"""

        test_inputs = [
            "Health advice for 75kg male, moderate activity",
            "Nutrition recommendations for female athlete",
            "Daily wellness plan for sedentary office worker"
        ]

        print("\n🧪 Testing clean model responses...")

        for i, test_input in enumerate(test_inputs):
            print(f"\n📝 Test {i+1}: {test_input}")

            format_instruction = """You are a wellness coach AI. You MUST respond in this EXACT format with REALISTIC numbers:

CRITICAL RULES:
- Water: Use 2-10 liters per DAY (never thousands)
- Protein: Use 50-200 grams per DAY (never thousands)
- Steps: Use 1,000-20,000 per DAY (never mix with nutrition)
- Sleep: Use 5-10 hours per NIGHT
- Always use appropriate units and realistic ranges

"""

            prompt = f"{format_instruction}Human: {test_input}\n\nAssistant:"
            inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=300,
                    temperature=0.2,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1,
                )

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = response.replace(prompt, "").strip()

            # Validate response quality
            quality_check = self.validate_response_quality(response)

            print(f"📊 Quality check: {quality_check}")
            print(f"📖 Response: {response[:200]}...")

    def validate_response_quality(self, response):
        """Check if response has realistic numbers and proper format"""

        issues = []

        # Check for unrealistic numbers
        if re.search(r'protein.*[0-9]{4,}', response, re.IGNORECASE):
            issues.append("❌ Unrealistic protein numbers")

        if re.search(r'water.*[0-9]{4,}', response, re.IGNORECASE):
            issues.append("❌ Unrealistic water numbers")

        if re.search(r'hydration.*[0-9]{4,}', response, re.IGNORECASE):
            issues.append("❌ Unrealistic hydration numbers")

        # Check for format compliance
        required_sections = [
            "## 1) Food Recommendation",
            "## 2) Physical Exercise",
            "## 3) Sleep and Social",
            "## 4) Overall Weekly"
        ]

        format_score = sum(1 for section in required_sections if any(part in response for part in section.split()))

        if format_score < 3:
            issues.append(f"❌ Format incomplete ({format_score}/4 sections)")

        if not issues:
            return "✅ Excellent - realistic numbers and proper format"
        else:
            return f"Issues found: {issues}"

def main():
    """Main training function"""

    model_name = "ContactDoctor/Bio-Medical-Llama-3-8B"

    print("🚀 CLEAN HEALTH COACH TRAINING SYSTEM")
    print("🎯 Goal: Train without number confusion from the start")
    print("=" * 60)

    try:
        # Initialize clean trainer
        trainer = CleanHealthCoachTrainer(model_name)

        # Train with clean data
        model, tokenizer = trainer.train_clean_model()

        if model is not None:
            # Test the clean model
            trainer.test_clean_model()
            print("\n✅ Clean health coach training complete!")
            print("📁 Model saved to './clean_health_coach'")
            print("🎉 No number confusion - realistic responses guaranteed!")
        else:
            print("❌ Training failed")

    except Exception as e:
        print(f"❌ Training error: {e}")
        print("💡 Try restarting runtime and running again")

if __name__ == "__main__":
    main()

print("\n💡 TRAINING FEATURES:")
print("✅ Realistic number ranges built-in")
print("✅ Proper format instruction on every example")
print("✅ Diverse training data (18+ examples)")
print("✅ Conservative training to prevent overfitting")
print("✅ Built-in quality validation")
print("✅ Memory-efficient for Colab")
print("\n🎯 Result: Health coach with sensible responses!")

In [ ]:
from huggingface_hub import upload_folder

def upload_trained_model():
    """Upload your trained model to replace the existing adapter"""

    print("📤 Uploading trained model to Hugging Face...")

    try:
        upload_folder(
            folder_path="./clean_health_coach",  # Your trained model
            repo_id="AnjaliNV/WellBeing_LLM",     # Your repository
            commit_message="Update with stronger template training",
            create_pr=False  # Direct push to main branch
        )

        print("✅ Upload successful!")
        print("🎯 Your repository now has the new trained adapter")

    except Exception as e:
        print(f"❌ Upload failed: {e}")

# Run it
upload_trained_model()

#test